In [1]:
!pip install transformers==4.43.3 -q
!pip install pyarrow==21.0.0 -q
!pip install peft==0.11.1 -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 73.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 64.9 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 31.6 MB/s eta 0:00:0000:01m00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk
from nltk.corpus import stopwords
from collections import Counter

# Download stopwords
nltk.download('stopwords')

# Load data
train = pd.read_csv('/kaggle/input/2025-sep-dl-gen-ai-project/train.csv')

emotion_cols = ['anger', 'fear', 'joy', 'sadness', 'surprise']
train.head()


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,text,anger,fear,joy,sadness,surprise,emotions
0,0,the dentist that did the work apparently did a...,1,0,0,1,0,['anger' 'sadness']
1,1,i'm gonna absolutely ~~suck~~ be terrible duri...,0,1,0,1,0,['fear' 'sadness']
2,2,"bridge: so leave me drowning calling houston, ...",0,1,0,1,0,['fear' 'sadness']
3,3,after that mess i went to see my now ex-girlfr...,1,1,0,1,0,['anger' 'fear' 'sadness']
4,4,"as he stumbled i ran off, afraid it might some...",0,1,0,0,0,['fear']


In [3]:
emotion_counts = train[emotion_cols].sum().sort_values(ascending=False)
most_common_emotion = emotion_counts.idxmax()
most_common_count = emotion_counts.max()

print("Most common emotion:", most_common_emotion)
print("Count:", most_common_count)


Most common emotion: fear
Count: 3860


In [4]:
train['label_count'] = train[emotion_cols].sum(axis=1)
count_exactly_2 = (train['label_count'] == 2).sum()
print("Instances with exactly 2 labels:", count_exactly_2)


Instances with exactly 2 labels: 2587


In [5]:
joy_sadness_together = ((train['joy'] == 1) & (train['sadness'] == 1)).sum()
print("Rows where Joy and Sadness occur together:", joy_sadness_together)


Rows where Joy and Sadness occur together: 96


In [6]:
surprise_pct = (train['surprise'].sum() / len(train)) * 100
print("Percentage containing 'Surprise':", round(surprise_pct, 2), "%")


Percentage containing 'Surprise': 29.28 %


In [7]:
max_diff = emotion_counts.max() - emotion_counts.min()
print("Maximum difference between any two emotion counts:", max_diff)


Maximum difference between any two emotion counts: 3052


In [8]:
train['word_count'] = train['text'].apply(lambda x: len(str(x).split()))
median_word_length = train['word_count'].median()
print("Median word length of texts:", median_word_length)


Median word length of texts: 13.0


In [9]:
corr_ang_fear = train['anger'].corr(train['fear'])
print("Correlation coefficient between anger and fear:", round(corr_ang_fear, 2))


Correlation coefficient between anger and fear: 0.08


In [10]:
before_chars = train['text'].apply(lambda x: len(str(x))).sum()

# Convert to lowercase
train['clean_text'] = train['text'].str.lower()

# Remove punctuation
train['clean_text'] = train['clean_text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

after_chars = train['clean_text'].apply(lambda x: len(str(x))).sum()

reduction_pct = ((before_chars - after_chars) / before_chars) * 100
print("Percentage reduction in total character count after removing punctuation:", round(reduction_pct, 2), "%")


Percentage reduction in total character count after removing punctuation: 3.26 %


In [11]:
stop_words = set(stopwords.words('english'))

# Get all words
all_words = [word for text in train['clean_text'] for word in text.split()]
unique_words = set(all_words)

stopword_overlap = unique_words.intersection(stop_words)
common_stopword_pct = (len(stopword_overlap) / len(unique_words)) * 100

print("Percentage of unique words that are stop words:", round(common_stopword_pct, 2), "%")


Percentage of unique words that are stop words: 1.54 %


In [12]:
filtered_words = [w for w in all_words if w not in stop_words and w.strip() != '']
word_freq = Counter(filtered_words)

most_common_words = word_freq.most_common(10)
fifth_most_freq = most_common_words[4][0]  # index 4 → 5th element

print("Top 10 words (excluding stopwords):", most_common_words)
print("5th most frequent word:", fifth_most_freq)


Top 10 words (excluding stopwords): [('head', 539), ('eyes', 438), ('like', 394), ('back', 365), ('heart', 334), ('one', 323), ('face', 293), ('get', 291), ('time', 271), ('still', 271)]
5th most frequent word: heart


In [13]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('stopwords')

# Load dataset
train = pd.read_csv('/kaggle/input/2025-sep-dl-gen-ai-project/train.csv')
emotion_cols = ['anger', 'fear', 'joy', 'sadness', 'surprise']


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
X = vectorizer.fit_transform(train['text'])
y = train[emotion_cols]

print("Q1) Feature Matrix Dimension:", X.shape)


Q1) Feature Matrix Dimension: (6827, 33896)


In [15]:
print("Q2) Feature representation type:", "Sparse" if hasattr(X, "toarray") else "Dense")


Q2) Feature representation type: Sparse


In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [17]:
dt = DecisionTreeClassifier(random_state=42, max_depth=6)
dt.fit(X_train, y_train)


DecisionTreeClassifier(max_depth=6, random_state=42)

In [18]:
n_nodes = dt.tree_.node_count
print("Q3) Number of nodes created:", n_nodes)


Q3) Number of nodes created: 55


In [19]:
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

train_macro_f1 = np.mean([
    f1_score(y_train.iloc[:,i], y_train_pred[:,i]) for i in range(len(emotion_cols))
])
test_macro_f1 = np.mean([
    f1_score(y_test.iloc[:,i], y_test_pred[:,i]) for i in range(len(emotion_cols))
])

print("Q4) Macro F1 on training split:", round(train_macro_f1, 4))
print("Q5) Macro F1 on test split:", round(test_macro_f1, 4))


Q4) Macro F1 on training split: 0.2147
Q5) Macro F1 on test split: 0.1976


In [20]:
test_data = pd.read_csv('/kaggle/input/2025-sep-dl-gen-ai-project/test.csv')
X_test_final = vectorizer.transform(test_data['text'])
y_test_pred_final = dt.predict(X_test_final)

submission = pd.DataFrame(y_test_pred_final, columns=emotion_cols)
submission.insert(0, 'id', test_data['id'])
submission.to_csv('decision_tree_tfidf_submission.csv', index=False)
submission.head()


,id,anger,fear,joy,sadness,surprise
0,0,0,1,0,0,0
1,1,0,1,0,0,0
2,2,0,1,0,0,0
3,3,0,1,0,0,0
4,4,0,1,0,0,0


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

sent_1 = "I am very happy and excited today!"
sent_2 = "I am feeling joyful and thrilled!"

tfidf = TfidfVectorizer(stop_words='english')
vectors = tfidf.fit_transform([sent_1, sent_2])

cos_sim = cosine_similarity(vectors[0], vectors[1])[0][0]
print("Q7) Cosine similarity:", round(cos_sim, 3))


Q7) Cosine similarity: 0.0


In [22]:
print("Q9) No — TF-IDF space is high-dimensional and sparse; KNN performs poorly.")


Q9) No — TF-IDF space is high-dimensional and sparse; KNN performs poorly.


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Select rows where anger or fear = 1
anger_fear_list = list(train[(train['anger'] == 1) | (train['fear'] == 1)]['text'])

# Pick two example sentences
sent_1 = anger_fear_list[0]
sent_2 = anger_fear_list[5]

print("Sentence 1:", sent_1)
print("Sentence 2:", sent_2)

# Compute TF-IDF vectors
tfidf = TfidfVectorizer(stop_words='english')
vectors = tfidf.fit_transform([sent_1, sent_2])

# Compute cosine similarity
cos_sim = cosine_similarity(vectors[0], vectors[1])[0][0]
print("\nQ7) Cosine Similarity:", round(cos_sim, 3))


Sentence 1: the dentist that did the work apparently did a lousy job as in just a few years my teeth decayed under the crowns so i had no choice but to get partials.
Sentence 2: i notice the small surprises and accidents in life and when pointed out, i get stares like i am a beast with leaves in my mouth.

Q7) Cosine Similarity: 0.0


In [24]:
import pandas as pd
import re
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch.utils.data import Dataset, DataLoader

# Load training data
train_df = pd.read_csv("/kaggle/input/2025-sep-dl-gen-ai-project/train.csv")

# Emotion labels (multi-label)
EMOTIONS = ["anger", "fear", "joy", "sadness", "surprise"]

# Simple text cleaning function
def clean_text(text):
    text = text.lower()                                           # convert to lowercase
    text = re.sub(r"http\S+", "", text)                          # remove URLs
    text = re.sub(r"[^a-zA-Z ]", " ", text)                      # remove punctuation and numbers
    text = re.sub(r"\s+", " ", text).strip()                     # remove extra spaces
    return text

# Apply cleaning
train_df["clean_text"] = train_df["text"].apply(clean_text)

# Train-validation split (80% training / 20% validation)
X_train, X_val, y_train, y_val = train_df["clean_text"], train_df[EMOTIONS], train_df[EMOTIONS], train_df[EMOTIONS]

X_train, X_val, y_train, y_val = train_test_split(
    train_df["clean_text"],
    train_df[EMOTIONS],
    test_size=0.2,
    random_state=42
)


In [25]:
from collections import defaultdict

# Build vocabulary for LSTM scratch model
def build_vocab(texts, min_freq=2):
    vocab = {"<pad>": 0}                        # index 0 reserved for padding
    freq = defaultdict(int)

    # Count frequency of each word
    for text in texts:
        for word in text.split():
            freq[word] += 1

    # Add words occurring ≥ min_freq
    for word, count in freq.items():
        if count >= min_freq:
            vocab[word] = len(vocab)

    return vocab

vocab = build_vocab(train_df["clean_text"])

# Tokenizer function → converts text into list of word IDs
def tokenizer_simple(text):
    return [vocab.get(word, 0) for word in text.split()]


In [26]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=100):
        self.texts = texts.tolist()
        self.labels = torch.tensor(labels.values, dtype=torch.float32)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize text → convert to sequence of word IDs
        token_ids = self.tokenizer(self.texts[idx])

        # Pad or trim sequence to fixed length
        token_ids = token_ids[:self.max_len] + [0] * (self.max_len - len(token_ids))

        return torch.tensor(token_ids), self.labels[idx]


In [27]:
import torch.nn as nn

class LSTMEmotion(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=128):
        super().__init__()

        # Convert word IDs → vectors
        self.embedding = nn.Embedding(vocab_size, embed_dim)

        # Bidirectional LSTM → captures left + right context
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)

        # Final layer outputs 5 emotion logits
        self.fc = nn.Linear(hidden_dim * 2, 5)

    def forward(self, x):
        x = self.embedding(x)                # shape: (batch, seq_len, embed_dim)
        _, (h, _) = self.lstm(x)             # h contains final hidden states (forward + backward)

        # Concatenate forward + backward LSTM hidden states
        h_final = torch.cat((h[0], h[1]), dim=1)

        # Predict emotions
        return self.fc(h_final)


In [28]:
import time
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# ---------------------------
# DEVICE SELECTION
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------------------
# MODEL DEFINITION
# ---------------------------
class LSTMEmotion(nn.Module):
    def __init__(self, vocab_size, embed_dim=64, hidden_dim=64, num_labels=5):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_labels)

    def forward(self, x):
        x = self.embed(x)
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out)

model = LSTMEmotion(len(vocab)).to(device)

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# ---------------------------
# DATASETS + LOADERS
# ---------------------------
train_ds = TextDataset(X_train, y_train, tokenizer_simple)
val_ds   = TextDataset(X_val, y_val, tokenizer_simple)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds, batch_size=64, num_workers=0)

# ---------------------------
# DEBUG: TEST THE DATALOADER
# ---------------------------
print("\nTesting if DataLoader works...")

for i, batch in enumerate(train_loader):
    print("✔ First batch loaded successfully!")
    break

# ---------------------------
# EVALUATION FUNCTION
# ---------------------------
def evaluate(model, loader):
    model.eval()
    preds, trues = [], []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            logits = model(x)
            prob = torch.sigmoid(logits)
            pred = (prob > 0.5).int()

            preds.append(pred.cpu())
            trues.append(y.cpu())

    preds = torch.cat(preds)
    trues = torch.cat(trues)

    return f1_score(trues, preds, average="macro")

# ---------------------------
# TRAINING LOOP
# ---------------------------
log_every = 40

for epoch in range(15):  # keep 3 epochs for test run
    model.train()
    print(f"\n🔵 Epoch {epoch+1} starting...")

    for step, (x, y) in enumerate(train_loader):
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        logits = model(x)
        loss = loss_fn(logits, y)
        loss.backward()
        optimizer.step()

        if step % log_every == 0:
            pct = (step+1) / len(train_loader) * 100
            print(f"  ➤ Step {step+1}/{len(train_loader)} ({pct:.1f}%) | Loss={loss.item():.4f}")

    f1 = evaluate(model, val_loader)
    print(f"✅ Epoch {epoch+1} — Validation Macro F1: {f1:.4f}")

# ---------------------------
# SAVE MODEL
# ---------------------------
torch.save(model.state_dict(), "lstm_model_fast.pth")
print("\n🎉 Training done! Model saved as lstm_model_fast.pth")


Using device: cuda

Testing if DataLoader works...
✔ First batch loaded successfully!

🔵 Epoch 1 starting...
  ➤ Step 1/86 (1.2%) | Loss=0.6787
  ➤ Step 41/86 (47.7%) | Loss=0.5640
  ➤ Step 81/86 (94.2%) | Loss=0.5595
✅ Epoch 1 — Validation Macro F1: 0.1427

🔵 Epoch 2 starting...
  ➤ Step 1/86 (1.2%) | Loss=0.5738
  ➤ Step 41/86 (47.7%) | Loss=0.5768
  ➤ Step 81/86 (94.2%) | Loss=0.5635
✅ Epoch 2 — Validation Macro F1: 0.1427

🔵 Epoch 3 starting...
  ➤ Step 1/86 (1.2%) | Loss=0.5649
  ➤ Step 41/86 (47.7%) | Loss=0.5844
  ➤ Step 81/86 (94.2%) | Loss=0.5968
✅ Epoch 3 — Validation Macro F1: 0.1427

🔵 Epoch 4 starting...
  ➤ Step 1/86 (1.2%) | Loss=0.5617
  ➤ Step 41/86 (47.7%) | Loss=0.5856
  ➤ Step 81/86 (94.2%) | Loss=0.5448
✅ Epoch 4 — Validation Macro F1: 0.1427

🔵 Epoch 5 starting...
  ➤ Step 1/86 (1.2%) | Loss=0.5949
  ➤ Step 41/86 (47.7%) | Loss=0.5919
  ➤ Step 81/86 (94.2%) | Loss=0.5622
✅ Epoch 5 — Validation Macro F1: 0.1427

🔵 Epoch 6 starting...
  ➤ Step 1/86 (1.2%) | Loss=0.5

In [29]:
# ============================
# BERT Multi-Label Emotion Classification
# ============================
import os
os.environ["TRANSFORMERS_NO_ADDITIONAL_CHAT_TEMPLATE"] = "1"
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset
from sklearn.metrics import f1_score

# ---------------------------
# DEVICE
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------------------
# PREPARE TOKENIZER
# ---------------------------
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize(batch):
    return tokenizer(
        batch["clean_text"],
        truncation=True,
        padding=False   # dynamic padding handled by DataCollator
    )

# ---------------------------
# METRICS FUNCTION
# ---------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))      # sigmoid for multi-label
    preds = (probs > 0.5).astype(int)      # threshold 0.5
    macro_f1 = f1_score(labels, preds, average="macro")
    return {"macro_f1": macro_f1}

# ---------------------------
# PREPARE DATASET
# ---------------------------
df2 = train_df.copy()
# convert labels → float for multi-label classification
df2["labels"] = df2[EMOTIONS].astype(float).values.tolist()

dataset = Dataset.from_pandas(df2)
dataset = dataset.train_test_split(test_size=0.2)
dataset = dataset.map(tokenize, batched=True)

# ---------------------------
# LOAD MODEL
# ---------------------------
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=5,
    problem_type="multi_label_classification"
)
model.to(device)

# ---------------------------
# TRAINING ARGUMENTS
# ---------------------------
args = TrainingArguments(
    output_dir="./bert",
    eval_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    fp16=True,          # mixed precision if GPU available
    report_to="none",
    run_name="bert-pretrained"
)

# ---------------------------
# DATA COLLATOR
# ---------------------------
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ---------------------------
# TRAINER
# ---------------------------
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# ---------------------------
# TRAIN MODEL
# ---------------------------
trainer.train()

# ---------------------------
# SAVE MODEL + TOKENIZER
# ---------------------------
model.save_pretrained("bert_model")
tokenizer.save_pretrained("bert_model")
print("\n🎉 Model and tokenizer saved successfully!")


2025-11-29 08:13:52.388244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764404032.766617      38 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764404032.877380      38 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Epoch,Training Loss,Validation Loss,Macro F1
1,0.410300,0.328169,0.736099
2,0.221700,0.277547,0.784020


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



🎉 Model and tokenizer saved successfully!


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import pandas as pd

# ------------------------------
# 1. Convert text → TF-IDF features
# ------------------------------
tfidf = TfidfVectorizer(
    max_features=12000,     # more features = better performance
    ngram_range=(1,2),      # include bigrams for better emotion capture
    stop_words='english'
)

X_tfidf = tfidf.fit_transform(train_df["clean_text"])

# ------------------------------
# 2. Train one classifier per emotion (multi-label)
# ------------------------------
models = {}

for emo in EMOTIONS:
    clf = LogisticRegression(
        max_iter=2000,         # increase iterations → more stable training
        class_weight="balanced", # handles imbalance in emotions
        solver="lbfgs"
    )
    
    clf.fit(X_tfidf, train_df[emo])
    models[emo] = clf

# ------------------------------
# 3. Predict on validation set
# ------------------------------
X_val_tfidf = tfidf.transform(X_val)

preds = {}
for emo in EMOTIONS:
    preds[emo] = models[emo].predict(X_val_tfidf)

preds_df = pd.DataFrame(preds)

# ------------------------------
# 4. Macro F1 Score
# ------------------------------
f1 = f1_score(y_val, preds_df, average="macro")
print("TF-IDF + Logistic Regression Macro F1:", f1)

TF-IDF + Logistic Regression Macro F1: 0.8877456926258276


In [31]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader

# ---------------------------
# DEVICE
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------------------
# LOAD TEST DATA
# ---------------------------
test_df = pd.read_csv("/kaggle/input/2025-sep-dl-gen-ai-project/test.csv")

# ---------------------------
# LOAD SAVED BERT MODEL
# ---------------------------
tokenizer = BertTokenizer.from_pretrained("bert_model")
model = BertForSequenceClassification.from_pretrained("bert_model")
model.to(device)
model.eval()

# ---------------------------
# TOKENIZE TEST DATA
# ---------------------------
# Dynamic padding for efficiency
encodings = tokenizer(
    list(test_df["text"]),
    truncation=True,
    padding=True,
    return_tensors="pt"
)

# Move input tensors to device
input_ids = encodings["input_ids"].to(device)
attention_mask = encodings["attention_mask"].to(device)

# ---------------------------
# BATCHED INFERENCE
# ---------------------------
batch_size = 32
predictions = []

with torch.no_grad():
    for i in range(0, len(test_df), batch_size):
        batch_input_ids = input_ids[i:i+batch_size]
        batch_attention_mask = attention_mask[i:i+batch_size]

        logits = model(
            input_ids=batch_input_ids,
            attention_mask=batch_attention_mask
        ).logits

        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs > 0.5).astype(int)

        predictions.extend(preds)

# ---------------------------
# CREATE SUBMISSION FILE
# ---------------------------
submission = pd.DataFrame(predictions, columns=EMOTIONS)
submission["id"] = test_df["id"]
submission = submission[["id"] + EMOTIONS]

# Save submission to Kaggle's working directory
submission.to_csv("/kaggle/working/submission.csv", index=False)
print("Submission saved successfully!")


Using device: cuda
Submission saved successfully!
